In [1]:
# install dependencies: 
!pip install pyyaml==5.1 'pycocotools>=2.0.1'
#!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
#!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 276kB 8.0MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=4234c3e53ab764a0b86c844bef9ccd62a2d12cb65b000e1b9267350377c959ca
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.7.0+cu101 True


In [2]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.6)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
     |████████████████████████████████| 6.6MB 12.6MB/s 
     |████████████████████████████████| 2.2MB 12.9MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.2.post20201218-cp36-none-any.whl size=40701 sha256=b9630a705ae0d79e1695a4161eaf4f29d0683b4f9af35e98cdcddb9b8ee582d6
  Stored in directory: /root/.cache/pip/wheels/b5/83/3f/e74a72d264351b5ada512da845a5c6abb81a34f6333c3a54ce
  Created wheel for iopath: filename=iopath-0.1.2-cp36-none-any.whl size=10508 sha256=4e9f8f299baa26d24020582eed7b81d32dae5abd4419520d5b26894c7ebf06ef
  Stored in directory: /root/.cache/pip/wheels/9e/01/e4/1b68f5a2a6b9450ea4246d91840a77e1169f7d4722d76bbc47
Successfully built fvcore iopath
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7

In [1]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [2]:
import time
from detectron2.utils.visualizer import ColorMode

import matplotlib.pyplot as plt
import matplotlib as mpl
import logging
import threading
import time
import math
from detectron2.data.datasets import register_coco_instances

In [3]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


To save a model, run the following

In [ ]:
#To save a model, run:
# !cp 'output/model_final.pth' 'drive/My Drive/CIS581/FinalProject/Ruleset/outputfile.pth'
#Do this directly after the training stage

For Loading a pretrained Model do the following (If takes more than 1 minute to run, check if you have the GPU enabled):

In [8]:
ls

drive/  sample_data/


In [13]:
videopath='/content/drive/My Drive/CIS581/cca96d47-c214cf4d.mp4'
outputpath="/content/drive/My Drive/CIS581/FinalProject/padestrians_solid.mp4"
rulesetpath="/drive/MyDrive/CIS581/Highres.pth"

In [14]:
#To load a model run:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 14
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
cfg.MODEL.WEIGHTS =rulesetpath# path to the model we trained
predictor = DefaultPredictor(cfg)


AssertionError: ignored

Running Live Video Detections 

In [ ]:
def threading_funct(frame):
  outputs = predictor(frame) 
  data=outputs["instances"].to("cpu")
  output_mask=data.pred_masks.numpy().astype(int)
  output_classes=data.pred_classes.numpy()
  count=output_classes.shape
  mask=output_mask*output_classes[:,np.newaxis, np.newaxis]
  #print("mask shape=",mask.shape)

  #print("max value=",max(mask))
  mask=np.where(mask==3,18,mask)
  #print("mask shape=",mask.shape)
  newimage1=np.amax(mask,axis=0)
  global value
  value=outputs["instances"].to("cpu").pred_boxes.tensor.numpy().astype(float)[3]
  global start
  start=(int((value[2]+value[0])/2),int((value[3]+value[1])/2))  
  global endpoint
  endpoint=(int(line_length*math.cos(np.deg2rad(arrow_angle)))+start[0],int(line_length*math.sin(np.deg2rad(arrow_angle)))+start[1])
  #Checking for Intersection
  line_y=100

  x_new=(math.cos(np.deg2rad(arrow_angle)))*10+start[0]
  global y_new, y1
  y_new=-(math.sin(np.deg2rad(arrow_angle))*10)+start[1]
  x1=start[0]
  y1=start[1]
  if (x1-x_new==0):
    x_new+=0.0000001
  if (y1-y_new==0):
    y_new+=0.000001  
  slope=((y1-y_new)/(x1-x_new))
  b=y1-(slope*x1)
  global intersect_loc_x
  intersect_loc_x=(100-b)/slope#(slope*line_y)+b
 
  global newimage
  newimage=newimage1
  global ready
  ready=True

  global newimageframe
  newimageframe=frame


print("set video fps before running this code")
print("Code may fail on first run, if this happens try running again.")
fps=29.97
print("fps=",fps)


#video=cv2.VideoCapture('480_360vid.mp4')
video=cv2.VideoCapture(videopath)

frame_width = int(video.get(3)*2) 
frame_height = int(video.get(4)) 
   
size = (frame_width, frame_height) 
output=cv2.VideoWriter(outputpath,  
                         cv2.VideoWriter_fourcc(*'Mp4V'), 
                         fps, size)
ready=True
counter=0
starttime=time.time()


arrow_angle=0
start=(0,0)
line_length=30
endpoint=(0,0)
intersect_loc_x=10000
minbound=0.3*(frame_width/2)
maxbound=(frame_width/2)-(0.3*(frame_width/2))

while(True):
  t=time.time()
  
  ret,frame=video.read()
 
  if ret==True:
    if ready==True:
      if counter==0:
        outputim=frame
      ready=False
      x=threading.Thread(target=threading_funct, args=(frame,) )
      x.start()   
    else:
      if counter==0:
        outputim=frame
        newimageframe=frame
      else:  
        outputim=(newimage*25).astype(np.uint8)
        outputim=cv2.cvtColor(outputim, cv2.COLOR_RGB2BGR)
        outputim=cv2.applyColorMap(outputim,cv2.COLORMAP_JET)  
    
    if counter==0:
      outputs = predictor(frame)
      newimageframe=frame
    arrow_angle+=0.7
    if arrow_angle>=360:
      arrow_angle=0

    outputimage=cv2.hconcat([frame,outputim])

    output.write(outputimage)
    counter=1
    timediff=time.time()-t

    if timediff < (1/fps):
     time.sleep((1.0/fps)-timediff)

  else:
    break
  

video.release()
output.release()
print("total Time=",time.time()-starttime)
print("Must check length of original video, if length is longer than")
print("total time, the program did not run in real time.")

set video fps before running this code
Code may fail on first run, if this happens try running again.
fps= 29.97
total Time= 0.0003116130828857422
Must check length of original video, if length is longer than
total time, the program did not run in real time.
